# Supabase (Postgres) | Supabase (Postgres)




> [Supabase](https://supabase.com/docs)はオープンソースのFirebaseの代替品です。`Supabase`は`PostgreSQL`の上に構築されており、強力なSQLクエリ機能を提供し、既存のツールやフレームワークとのシンプルなインターフェースを実現しています。
>
> > [Supabase](https://supabase.com/docs) is an open-source Firebase alternative. `Supabase` is built on top of `PostgreSQL`, which offers strong SQL querying capabilities and enables a simple interface with already-existing tools and frameworks.

> [PostgreSQL](https://en.wikipedia.org/wiki/PostgreSQL)（通称 `Postgres`）は、拡張性とSQL準拠を重視した無料のオープンソースのリレーショナルデータベース管理システム（RDBMS）です。
>
> > [PostgreSQL](https://en.wikipedia.org/wiki/PostgreSQL) also known as `Postgres`, is a free and open-source relational database management system (RDBMS) emphasizing extensibility and SQL compliance.

このノートブックは、`Supabase`と`pgvector`をVectorStoreとして使用する方法を示しています。

> This notebook shows how to use `Supabase` and `pgvector` as your VectorStore.

このノートブックを実行するには、以下を確認してください：

> To run this notebook, please ensure:

* `pgvector` 拡張機能が有効になっています

  > the `pgvector` extension is enabled

* `supabase-py`パッケージがインストールされている必要があります

  > you have installed the `supabase-py` package

* データベースに`match_documents`関数を作成したこと

  > that you have created a `match_documents` function in your database

* `public`スキーマに、以下に示すような`documents`テーブルがあること。

  > that you have a `documents` table in your `public` schema similar to the one below.


以下の関数はコサイン類似度を決定しますが、必要に応じて調整することができます。

> The following function determines cosine similarity, but you can adjust to your needs.

```sql
-- Enable the pgvector extension to work with embedding vectors
create extension if not exists vector;

-- Create a table to store your documents
create table
  documents (
    id uuid primary key,
    content text, -- corresponds to Document.pageContent
    metadata jsonb, -- corresponds to Document.metadata
    embedding vector (1536) -- 1536 works for OpenAI embeddings, change if needed
  );

-- Create a function to search for documents
create function match_documents (
  query_embedding vector (1536),
  filter jsonb default '{}'
) returns table (
  id uuid,
  content text,
  metadata jsonb,
  similarity float
) language plpgsql as $$
#variable_conflict use_column
begin
  return query
  select
    id,
    content,
    metadata,
    1 - (documents.embedding <=> query_embedding) as similarity
  from documents
  where metadata @> filter
  order by documents.embedding <=> query_embedding;
end;
$$;
```




In [ ]:
# with pip
!pip install supabase

# with conda
# !conda install -c conda-forge supabase

`OpenAIEmbeddings`を使用するためには、OpenAIのAPIキーを取得する必要があります。

> We want to use `OpenAIEmbeddings` so we have to get the OpenAI API Key.




In [15]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

In [16]:
os.environ["SUPABASE_URL"] = getpass.getpass("Supabase URL:")

In [17]:
os.environ["SUPABASE_SERVICE_KEY"] = getpass.getpass("Supabase Service Key:")

In [ ]:
# If you're storing your Supabase and OpenAI API keys in a .env file, you can load them with dotenv
from dotenv import load_dotenv

load_dotenv()

まず、Supabaseクライアントを作成し、OpenAIのembeddingsクラスのインスタンスを生成します。

> First we'll create a Supabase client and instantiate a OpenAI embeddings class.




In [19]:
import os

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import SupabaseVectorStore
from supabase.client import Client, create_client

supabase_url = os.environ.get("SUPABASE_URL")
supabase_key = os.environ.get("SUPABASE_SERVICE_KEY")
supabase: Client = create_client(supabase_url, supabase_key)

embeddings = OpenAIEmbeddings()

次に、ベクターストア用のデータを読み込んでパースします（データベースに既に埋め込み付きのドキュメントが保存されている場合は、この手順をスキップしてください）。

> Next we'll load and parse some data for our vector store (skip if you already have documents with embeddings stored in your DB).




In [20]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

loader = TextLoader("../../modules/state_of_the_union.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

上記のドキュメントをデータベースに挿入してください。各ドキュメントに対して埋め込みが自動的に生成されます。ドキュメントの量に応じて、chunk\_sizeを調整することができます。デフォルトは500ですが、必要に応じてそれを下げることが必要になるかもしれません。

> Insert the above documents into the database. Embeddings will automatically be generated for each document. You can adjust the chunk\_size based on the amount of documents you have. The default is 500 but lowering it may be necessary.




In [6]:
vector_store = SupabaseVectorStore.from_documents(
    docs,
    embeddings,
    client=supabase,
    table_name="documents",
    query_name="match_documents",
    chunk_size=500,
)

また、データベースに既に埋め込みを含むドキュメントがある場合は、新しい `SupabaseVectorStore` を直接インスタンス化してください：

> Alternatively if you already have documents with embeddings in your database, simply instantiate a new `SupabaseVectorStore` directly:




In [10]:
vector_store = SupabaseVectorStore(
    embedding=embeddings,
    client=supabase,
    table_name="documents",
    query_name="match_documents",
)

最後に、類似性検索を実行してテストしてみましょう：

> Finally, test it out by performing a similarity search:




In [ ]:
query = "What did the president say about Ketanji Brown Jackson"
matched_docs = vector_store.similarity_search(query)

In [ ]:
print(matched_docs[0].page_content)

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


## Similarity search with score | スコア付き類似性検索




返される距離スコアはコサイン距離です。したがって、スコアが低い方が良いです。

> The returned distance score is cosine distance. Therefore, a lower score is better.




In [9]:
matched_docs = vector_store.similarity_search_with_relevance_scores(query)

In [10]:
matched_docs[0]

(Document(page_content='Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. \n\nTonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.', metadata={'source': '../../../state_of_the_union.txt'}),
 0.802509746274066)

## Retriever options | リトリバーのオプション

このセクションでは、SupabaseVectorStoreをリトリーバーとして使用するさまざまなオプションについて説明します。

> This section goes over different options for how to use SupabaseVectorStore as a retriever.

### Maximal Marginal Relevance Searches | 最大限余剰関連性検索

リトリバーオブジェクトで類似性検索を使用することに加えて、`mmr`も使用できます。

> In addition to using similarity search in the retriever object, you can also use `mmr`.




In [11]:
retriever = vector_store.as_retriever(search_type="mmr")

In [12]:
matched_docs = retriever.get_relevant_documents(query)

In [13]:
for i, d in enumerate(matched_docs):
    print(f"\n## Document {i}\n")
    print(d.page_content)


## Document 0

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.

## Document 1

One was stationed at bases and breathing in toxic smoke from “burn pits” that incinerated wastes of war—medical and hazard material, jet fuel, and more. 

When they came home, many